In [1]:
# -*- coding: utf-8 -*-

"""
EVCのためのEV-GMMを構築します. そして, 適応学習する.
詳細 : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580aabf534c4dbb8bc.pdf

This program make EV-GMM for EVC. Then, it make adaptation learning.
Check detail : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580abf534c4dbb8bc.pdf
"""

'\nEVCは声質変換の一つの手法です.\n詳細 : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580abf534c4dbb8bc.pdf\n\nEVC is kind of Voice Conversion(VC).\nCheck detail : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580abf534c4dbb8bc.pdf\n'

In [2]:
from __future__ import division, print_function

import os
from shutil import rmtree
import argparse
import glob
import pickle
import time

import numpy as np
from numpy.linalg import norm 
from sklearn.decomposition import PCA
from sklearn.mixture import GMM # sklearn 0.20.0から使えない
from sklearn.preprocessing import StandardScaler
import scipy.signal
import scipy.sparse

%matplotlib inline
import matplotlib.pyplot as plt
import IPython 
from IPython.display import Audio 

import soundfile as sf
import wave 
import pyworld as pw
import librosa.display

from dtw import dtw
import warnings
warnings.filterwarnings('ignore')

In [3]:
"""
Parameters

__Mixtured : GMM混合数
__versions : 実験セット
__convert_source : 変換元話者のパス
__convert_target : 変換先話者のパス
"""
# parameters 
__Mixtured = 40
__versions = 'pre-stored0.1.1'
__convert_source = 'input/EJM10/V01/T01/TIMIT/000/*.wav' 
__convert_target = 'adaptation/EJM04/V01/T01/ATR503/A/*.wav'

# settings
__same_path = './utterance/' + __versions + '/'

Mixtured = __Mixtured
pre_stored_pickle = __same_path + __versions + '.pickle'
pre_stored_source_list = __same_path + 'pre-source/**/V01/T01/**/*.wav'
pre_stored_list = __same_path + "pre/**/V01/T01/**/*.wav"
#pre_stored_target_list = "" (not yet)
pre_stored_gmm_init_pickle = __same_path + __versions + '_init-gmm.pickle'
pre_stored_sv_npy = __same_path + __versions + '_sv.npy'

save_for_evgmm_covarXX = __same_path + __versions + '_covarXX.npy'
save_for_evgmm_covarYX = __same_path + __versions + '_covarYX.npy'
save_for_evgmm_fitted_source = __same_path + __versions + '_fitted_source.npy'
save_for_evgmm_fitted_target = __same_path + __versions + '_fitted_target.npy'
save_for_evgmm_weights = __same_path + __versions + '_weights.npy'
save_for_evgmm_source_means = __same_path + __versions + '_source_means.npy'

for_convert_source = __same_path + __convert_source
for_convert_target = __same_path + __convert_target
converted_voice_npy = __same_path + 'sp_converted_' + __versions  
converted_voice_wav = __same_path + 'sp_converted_' + __versions 
mfcc_save_fig_png = __same_path + 'mfcc3dim_' + __versions 
f0_save_fig_png = __same_path + 'f0_converted' + __versions
converted_voice_with_f0_wav = __same_path + 'sp_f0_converted' + __versions

In [4]:
EPSILON = 1e-8

class MFCC:
    """
    MFCC() : メル周波数ケプストラム係数(MFCC)を求めたり、MFCCからスペクトルに変換したりするクラス.
    動的特徴量(delta)が実装途中.
    ref : http://aidiary.hatenablog.com/entry/20120225/1330179868
    """
    
    
    def __init__(self, frequency, nfft=1026, dimension=24, channels=24):
        """
        各種パラメータのセット
        nfft : FFTのサンプル点数
        frequency : サンプリング周波数
        dimension : MFCC次元数
        channles : メルフィルタバンクのチャンネル数(dimensionに依存)
        fscale : 周波数スケール軸
        filterbankl, fcenters : フィルタバンク行列, フィルタバンクの頂点(?)
        """
        self.nfft = nfft
        self.frequency = frequency
        self.dimension = dimension
        self.channels = channels
        self.fscale = np.fft.fftfreq(self.nfft, d = 1.0 / self.frequency)[: int(self.nfft / 2)]
        self.filterbank, self.fcenters = self.melFilterBank()
    
    def hz2mel(self, f):
        """
        周波数からメル周波数に変換
        """
        return 1127.01048 * np.log(f / 700.0 + 1.0)
    
    def mel2hz(self, m):
        """
        メル周波数から周波数に変換
        """     
        return 700.0 * (np.exp(m / 1127.01048) - 1.0)

    def melFilterBank(self):
        """
        メルフィルタバンクを生成する
        """      
        fmax = self.frequency / 2
        melmax = self.hz2mel(fmax)
        nmax = int(self.nfft / 2)
        df = self.frequency / self.nfft
        dmel = melmax / (self.channels + 1)
        melcenters = np.arange(1, self.channels + 1) * dmel
        fcenters = self.mel2hz(melcenters)
        indexcenter = np.round(fcenters / df)
        indexstart = np.hstack(([0], indexcenter[0:self.channels - 1]))
        indexstop = np.hstack((indexcenter[1:self.channels], [nmax]))

        filterbank = np.zeros((self.channels, nmax))
        for c in np.arange(0, self.channels):
            increment = 1.0 / (indexcenter[c] - indexstart[c])
            # np,int_ は np.arangeが[0. 1. 2. ..]となるのをintにする
            for i in np.int_(np.arange(indexstart[c], indexcenter[c])):
                filterbank[c, i] = (i - indexstart[c]) * increment
            decrement = 1.0 / (indexstop[c] - indexcenter[c])
            # np,int_ は np.arangeが[0. 1. 2. ..]となるのをintにする
            for i in np.int_(np.arange(indexcenter[c], indexstop[c])):
                filterbank[c, i] = 1.0 - ((i - indexcenter[c]) * decrement)

        return filterbank, fcenters
    
    def mfcc(self, spectrum):
        """
        スペクトルからMFCCを求める.
        """
        mspec = []
        mspec = np.log10(np.dot(spectrum, self.filterbank.T))
        mspec = np.array(mspec)
        
        return scipy.fftpack.realtransforms.dct(mspec, type=2, norm="ortho", axis=-1)
    
    def delta(self, mfcc):
        """
        MFCCから動的特徴量を求める.
        現在は,求める特徴量フレームtをt-1とt+1の平均としている.
        """
        mfcc = np.concatenate([
            [mfcc[0]], 
            mfcc, 
            [mfcc[-1]]
        ]) # 最初のフレームを最初に、最後のフレームを最後に付け足す
        delta = None
        for i in range(1, mfcc.shape[0] - 1):
            slope = (mfcc[i+1] - mfcc[i-1]) / 2
            if delta is None:
                delta = slope
            else:
                delta = np.vstack([delta, slope])
        return delta
    
    def imfcc(self, mfcc, spectrogram):
        """
        MFCCからスペクトルを求める.
        """
        im_sp = np.array([])
        for i in range(mfcc.shape[0]):
            mfcc_s = np.hstack([mfcc[i], [0] * (self.channels - self.dimension)])
            mspectrum = scipy.fftpack.idct(mfcc_s, norm='ortho')
            # splrep はスプライン補間のための補間関数を求める
            tck = scipy.interpolate.splrep(self.fcenters, np.power(10, mspectrum))
            # splev は指定座標での補間値を求める
            im_spectrogram = scipy.interpolate.splev(self.fscale, tck)
            im_sp = np.concatenate((im_sp, im_spectrogram), axis=0)
            
        return im_sp.reshape(spectrogram.shape)
            
    def trim_zeros_frames(x, eps=1e-7):
        """
        無音区間を取り除く.
        """
        T, D = x.shape
        s = np.sum(np.abs(x), axis=1)
        s[s < 1e-7] = 0.
        return x[s > eps]

In [5]:
def analyse_by_world_with_harverst(x, fs):
    """
    WORLD音声分析合成器で基本周波数F0,スペクトル包絡,非周期成分を求める.
    基本周波数F0についてはharvest法により,より精度良く求める.
    """
    # 4 Harvest with F0 refinement (using Stonemask)
    frame_period = 5
    _f0_h, t_h = pw.harvest(x, fs, frame_period=frame_period)
    f0_h = pw.stonemask(x, _f0_h, t_h, fs)
    sp_h = pw.cheaptrick(x, f0_h, t_h, fs)
    ap_h = pw.d4c(x, f0_h, t_h, fs)
    
    return f0_h, sp_h, ap_h

def wavread(file):
    """
    wavファイルから音声トラックとサンプリング周波数を抽出する.
    """
    wf = wave.open(file, "r")
    fs = wf.getframerate()
    x = wf.readframes(wf.getnframes())
    x = np.frombuffer(x, dtype= "int16") / 32768.0
    wf.close()
    return x, float(fs)

def preEmphasis(signal, p=0.97):
    """
    MFCC抽出のための高域強調フィルタ.
    波形を通すことで,高域成分が強調される.
    """
    return scipy.signal.lfilter([1.0, -p], 1, signal)

def alignment(source, target, path):
    """
    タイムアライメントを取る.
    target音声をsource音声の長さに合うように調整する.
    """
    # ここでは814に合わせよう(targetに合わせる)
    # p_p = 0 if source.shape[0] > target.shape[0] else 1

    #shapes = source.shape if source.shape[0] > target.shape[0] else target.shape 
    shapes = source.shape
    align = np.array([])
    for (i, p) in enumerate(path[0]):
        if i != 0:
            if j != p:
                temp = np.array(target[path[1][i]])
                align = np.concatenate((align, temp), axis=0)
        else:
            temp = np.array(target[path[1][i]])
            align = np.concatenate((align, temp), axis=0)   
        
        j = p
        
    return align.reshape(shapes)

In [6]:
"""
pre-stored学習のためのパラレル学習データを作る。
時間がかかるため、利用できるlearn-data.pickleがある場合はそれを利用する。
それがない場合は一から作り直す。
"""
timer_start = time.time()
if os.path.exists(pre_stored_pickle):
    print("exist, ", pre_stored_pickle)
    with open(pre_stored_pickle, mode='rb') as f:
        total_data = pickle.load(f)
        print("open, ", pre_stored_pickle)
        print("Load pre-stored time = ", time.time() - timer_start , "[sec]")
else:
    source_mfcc = []
    #source_data_sets = []
    for name in sorted(glob.iglob(pre_stored_source_list, recursive=True)):
        print(name)
        x, fs = sf.read(name)
        f0, sp, ap = analyse_by_world_with_harverst(x, fs)
    
        mfcc = MFCC(fs)
        source_mfcc_temp = mfcc.mfcc(sp)
        #source_data = np.hstack([source_mfcc_temp, mfcc.delta(source_mfcc_temp)]) # static & dynamic featuers
        source_mfcc.append(source_mfcc_temp)
        #source_data_sets.append(source_data)

    total_data = []
    
    i = 0
    _s_len = len(source_mfcc)
    for name in sorted(glob.iglob(pre_stored_list, recursive=True)):
        print(name, len(total_data))
        x, fs = sf.read(name)
        f0, sp, ap = analyse_by_world_with_harverst(x, fs)

        mfcc = MFCC(fs)
        target_mfcc = mfcc.mfcc(sp)

        dist, cost, acc, path = dtw(source_mfcc[i%_s_len], target_mfcc, dist=lambda x, y: norm(x - y, ord=1))
        #print('Normalized distance between the two sounds:' + str(dist))
        #print("target_mfcc = {0}".format(target_mfcc.shape))

        aligned = alignment(source_mfcc[i%_s_len], target_mfcc, path)
        #target_data_sets = np.hstack([aligned, mfcc.delta(aligned)]) # static & dynamic features
        #learn_data = np.hstack((source_data_sets[i], target_data_sets))
        learn_data = np.hstack([source_mfcc[i%_s_len], aligned])

        total_data.append(learn_data)
        i += 1
        
    with open(pre_stored_pickle, 'wb') as output:
        pickle.dump(total_data, output)
        print("Make, ", pre_stored_pickle)
        print("Make pre-stored time = ", time.time() - timer_start , "[sec]")

exist,  ./utterance/pre-stored0.1.1/pre-stored0.1.1.pickle
open,  ./utterance/pre-stored0.1.1/pre-stored0.1.1.pickle
Load pre-stored time =  2.1454219818115234 [sec]


In [7]:
"""
全事前学習出力話者からラムダを推定する.
ラムダは適応学習で変容する.
"""

S = len(total_data)
D = int(total_data[0].shape[1] / 2)
print("total_data[0].shape = ", total_data[0].shape)
print("S = ", S)
print("D = ", D)

timer_start = time.time()
if os.path.exists(pre_stored_gmm_init_pickle):
    print("exist, ", pre_stored_gmm_init_pickle)
    with open(pre_stored_gmm_init_pickle, mode='rb') as f:
        initial_gmm = pickle.load(f)
        print("open, ", pre_stored_gmm_init_pickle)
        print("Load initial_gmm time = ", time.time() - timer_start , "[sec]")  
else:
    initial_gmm = GMM(n_components = Mixtured, covariance_type = 'full')
    initial_gmm.fit(np.vstack(total_data))
    with open(pre_stored_gmm_init_pickle, 'wb') as output:
        pickle.dump(initial_gmm, output)
        print("Make, ", initial_gmm)
        print("Make initial_gmm time = ", time.time() - timer_start , "[sec]")    
            
weights = initial_gmm.weights_
source_means = initial_gmm.means_[:, :D]
target_means = initial_gmm.means_[:, D:]
covarXX = initial_gmm.covars_[:, :D, :D]
covarXY = initial_gmm.covars_[:, :D, D:]
covarYX = initial_gmm.covars_[:, D:, :D]
covarYY = initial_gmm.covars_[:, D:, D:]

fitted_source = source_means
fitted_target = target_means

total_data[0].shape =  (959, 48)
S =  1012
D =  24
exist,  ./utterance/pre-stored0.1.1/pre-stored0.1.1_init-gmm.pickle
open,  ./utterance/pre-stored0.1.1/pre-stored0.1.1_init-gmm.pickle
Load initial_gmm time =  0.19963502883911133 [sec]


In [8]:
"""
SVはGMMスーパーベクトルで、各pre-stored学習における出力話者について平均ベクトルを推定する。
GMMの学習を見てみる必要があるか？
"""

timer_start = time.time()
if os.path.exists(pre_stored_sv_npy):
    print("exist, ", pre_stored_sv_npy)
    sv = np.load(pre_stored_sv_npy)
    print("open, ", pre_stored_sv_npy)
    print("Load pre_stored_sv time = ", time.time() - timer_start , "[sec]")  
    
else:
    sv = []
    for i in range(S):
        gmm = GMM(n_components = Mixtured, params = 'm', init_params = '', covariance_type = 'full')
        gmm.weights_ = initial_gmm.weights_
        gmm.means_ = initial_gmm.means_
        gmm.covars_ = initial_gmm.covars_
        gmm.fit(total_data[i])
        sv.append(gmm.means_)
    sv = np.array(sv)
    np.save(pre_stored_sv_npy, sv)
    print("Make pre_stored_sv time = ", time.time() - timer_start , "[sec]")  

exist,  ./utterance/pre-stored0.1.1/pre-stored0.1.1_sv.npy
open,  ./utterance/pre-stored0.1.1/pre-stored0.1.1_sv.npy
Load pre_stored_sv time =  0.3458998203277588 [sec]


In [9]:
"""
各事前学習出力話者のGMM平均ベクトルに対して主成分分析(PCA)を行う.
PCAで求めた固有値と固有ベクトルからeigenvectorsとbiasvectorsを作る.
"""
timer_start = time.time()
#source_pca
source_n_component, source_n_features = sv[:, :, :D].reshape(S, Mixtured*D).shape
# 標準化(分散を1、平均を0にする)
source_stdsc = StandardScaler()
# 共分散行列を求める
source_X_std = source_stdsc.fit_transform(sv[:, :, :D].reshape(S, Mixtured*D)) 

# PCAを行う
source_cov = source_X_std.T @ source_X_std / (source_n_component - 1)
source_W, source_V_pca = np.linalg.eig(source_cov)

print(source_W.shape)
print(source_V_pca.shape)

# データを主成分の空間に変換する
source_X_pca = source_X_std @ source_V_pca
print(source_X_pca.shape)

#target_pca
target_n_component, target_n_features = sv[:, :, D:].reshape(S, Mixtured*D).shape
# 標準化(分散を1、平均を0にする)
target_stdsc = StandardScaler()
#共分散行列を求める
target_X_std = target_stdsc.fit_transform(sv[:, :, D:].reshape(S, Mixtured*D)) 

#PCAを行う
target_cov = target_X_std.T @ target_X_std / (target_n_component - 1)
target_W, target_V_pca = np.linalg.eig(target_cov)

print(target_W.shape)
print(target_V_pca.shape)

# データを主成分の空間に変換する
target_X_pca = target_X_std @ target_V_pca
print(target_X_pca.shape)

eigenvectors = source_X_pca.reshape((Mixtured, D, S)), target_X_pca.reshape((Mixtured, D, S))
source_bias = np.mean(sv[:, :, :D], axis=0)
target_bias = np.mean(sv[:, :, D:], axis=0)
biasvectors = source_bias.reshape((Mixtured, D)), target_bias.reshape((Mixtured, D))

print("Do PCA time = ", time.time() - timer_start , "[sec]")  

(960,)
(960, 960)
(1012, 960)
(960,)
(960, 960)
(1012, 960)
Do PCA time =  5.670940160751343 [sec]


In [10]:
"""
声質変換に用いる変換元音声と目標音声を読み込む.
"""

timer_start = time.time()
source_mfcc_for_convert = []
source_sp_for_convert = []
source_f0_for_convert = []
source_ap_for_convert = []
fs_source = None
for name in sorted(glob.iglob(for_convert_source, recursive=True)):
    print("source = ", name)
    x_source, fs_source = sf.read(name)
    f0_source, sp_source, ap_source = analyse_by_world_with_harverst(x_source, fs_source)
    mfcc_source = MFCC(fs_source)
    #mfcc_s_tmp = mfcc_s.mfcc(sp)
    #source_mfcc_for_convert = np.hstack([mfcc_s_tmp, mfcc_s.delta(mfcc_s_tmp)])
    source_mfcc_for_convert.append(mfcc_source.mfcc(sp_source))
    source_sp_for_convert.append(sp_source)
    source_f0_for_convert.append(f0_source)
    source_ap_for_convert.append(ap_source)

target_mfcc_for_fit = []
target_f0_for_fit = []
target_ap_for_fit = []
for name in sorted(glob.iglob(for_convert_target, recursive=True)):
    print("target = ", name)
    x_target, fs_target = sf.read(name)
    f0_target, sp_target, ap_target = analyse_by_world_with_harverst(x_target, fs_target)
    mfcc_target = MFCC(fs_target)
    #mfcc_target_tmp = mfcc_target.mfcc(sp_target)
    #target_mfcc_for_fit = np.hstack([mfcc_t_tmp, mfcc_t.delta(mfcc_t_tmp)])
    target_mfcc_for_fit.append(mfcc_target.mfcc(sp_target))
    target_f0_for_fit.append(f0_target)
    target_ap_for_fit.append(ap_target)

# 全部numpy.arrrayにしておく
source_data_mfcc = np.array(source_mfcc_for_convert)
source_data_sp = np.array(source_sp_for_convert)
source_data_f0 = np.array(source_f0_for_convert)
source_data_ap = np.array(source_ap_for_convert)

target_mfcc = np.array(target_mfcc_for_fit)
target_f0 = np.array(target_f0_for_fit)
target_ap = np.array(target_ap_for_fit)

print("Load Input and Target Voice time = ", time.time() - timer_start , "[sec]")  

source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A11.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A14.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A17.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A18.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A19.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A20.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A21.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A22.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A23.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A24.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A25.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A26.wav
source =  ./utterance/pre-stored0.1.1/input/EJM10/V01/T01/TIMIT/000/A27.wav
source =  ./

In [1]:
"""
適応話者学習を行う.
つまり,事前学習出力話者から目標話者の空間を作りだす.

適応話者文数ごとにfitted_targetを集めるのは未実装.
"""

timer_start = time.time()
epoch=1000

py = GMM(n_components = Mixtured, covariance_type = 'full')
py.weights_ = weights
py.means_ = target_means
py.covars_ = covarYY

fitted_target = None

for i in range(len(target_mfcc)):
    print("adaptation = ", i+1, "/", len(target_mfcc))
    target = target_mfcc[i]

    for x in range(epoch):
        print("epoch = ", x)
        predict = py.predict_proba(np.atleast_2d(target))
        y = np.sum([predict[:, i: i + 1] * (target - biasvectors[1][i])
                    for i in range(Mixtured)], axis = 1)
        gamma = np.sum(predict, axis = 0)

        left = np.sum([gamma[i] * np.dot(eigenvectors[1][i].T,
                                         np.linalg.solve(py.covars_, eigenvectors[1])[i])
                       for i in range(Mixtured)], axis=0)
        right = np.sum([np.dot(eigenvectors[1][i].T, 
                               np.linalg.solve(py.covars_, y)[i]) 
                        for i in range(Mixtured)], axis = 0)
        weight = np.linalg.solve(left, right)

        fitted_target = np.dot(eigenvectors[1], weight) + biasvectors[1]
        py.means_ = fitted_target
        
print("Load Input and Target Voice time = ", time.time() - timer_start , "[sec]")  

NameError: name 'time' is not defined

In [ ]:
"""
変換に必要なものを残しておく.
"""
np.save(save_for_evgmm_covarXX, covarXX)
np.save(save_for_evgmm_covarYX, covarYX)
np.save(save_for_evgmm_fitted_source, fitted_source)
np.save(save_for_evgmm_fitted_target, fitted_target)
np.save(save_for_evgmm_weights, weights)
np.save(save_for_evgmm_source_means, source_means)